#### Dependencies

In [33]:
import numpy as np
import csv
import os
import sys  # Import the sys module
import time
from functools import partial
from multiprocessing import Pool, cpu_count
from scipy.optimize import differential_evolution

sys.path.append('..')
from dataset_reader import Traces_Dataset
from DE_obj_model import de_obj_model   
from exp_hh_model import HH_model_exp

#### Read test dataset

In [2]:
dataset = Traces_Dataset('../dataset_test.csv')

params = dataset.params.numpy()
current_traces = dataset.current_traces.numpy()
time_traces = dataset.time_traces.numpy()

sample = 32
prestep_V_2d = dataset.prestep_V[sample].numpy().reshape(-1,1)
step_Vs_2d = dataset.step_Vs[sample].numpy().reshape(-1,1)
t = time_traces[sample]

target_traces = current_traces[sample]
target_params = params[sample]

In [3]:
# sim setup for obj evaluation model
sim_setup_2d = {'prestep_V': prestep_V_2d, 'step_Vs': step_Vs_2d, 't': t}

# model = de_obj_model(target_params, sim_setup_2d)

In [27]:
# sim_traces = model.simulation()
# sim_traces.shape

#### Plotting function for trail, target, and real data

In [8]:
def plot_trail_target_real(sim_setup_1d, t, sim_traces, target_traces, target_params): 
    '''
    sim_setup_1d is the exp setup for hh model, all default as in hh class
    t is the target time traces from generated dataset
    sim_traces is the generated during searching stage
    target_traces is the target current traces from the generated dataset
    target_params is the target params from the generated dataset, dict
    '''
    # compare samples and simulations using sample params
    hh_model = HH_model_exp(target_params, sim_setup_1d)
    current_traces_sim = hh_model.simulation()

    colors = ['blue', 'red', 'green', 'purple', 'orange', 'yellow', 'cyan', 'magenta', 'brown', 'gray', 'black']

    for step in range(t.shape[0]): 
        plt.plot(t[step], sim_traces[step], linestyle='-', color=colors[step])
        plt.plot(t[step], target_traces[step], linestyle=':', color=colors[step])
        plt.plot(sim_setup_1d['t'], current_traces_sim[step], linestyle='--', color=colors[step])
    plt.legend()
    plt.title(f"prestep_V: {sim_setup_1d['prestep_V']}; step_V1: {sim_setup_1d['step_Vs'][0]}")


In [9]:
# sim setup for plotting real data
sim_setup_1d = {'prestep_V': np.squeeze(prestep_V_2d), 'step_Vs': np.squeeze(step_Vs_2d), 't': np.arange(0.0, 6.0, 0.01)}
target_params_dict = {'p': target_params[0], 'g_max': target_params[1], 'E_rev': target_params[2], 'a_m': target_params[3], 'b_m': target_params[4], 'delta_m': target_params[5], 's_m': target_params[6]}
plot_trail_target_real(sim_setup_1d, t, sim_traces, target_traces, target_params_dict)

NameError: name 'sim_traces' is not defined

#### Define obj function

In [4]:
def obj(x, *args): 
    '''
    x: a 1-D array of the variables for the obj function (the parameters we are estimating)
    args: a tupleo f additional fixed parameters (prestep_V, step_V0, time_traces)
    *args=(sim_setup_2d, target_current_trances)
    '''
    trail_model = de_obj_model(x, args[0])
    trail_traces = trail_model.simulation()
    # print(trail_traces[1])
    target_model = de_obj_model(args[1], args[0])
    target_traces = target_model.simulation()
    # print(target_traces[1]) 

    fit = np.sum(np.square(trail_traces - target_traces))
    # relative_error = fit/np.sum(np.square(target_traces))
    
    return fit


In [5]:
fitness = obj(params[32], *(sim_setup_2d, target_params))
print(fitness)

0.0


#### Define bounds

In [6]:
# these bounds are from the distribution of the params in the dataset used for NN training
params_searching_bounds = {
    'p': (1, 4),
    'g_max': (100, 140), 
    'E_rev': (-100, -60), 
    'a_m': (0, 100), 
    'b_m': (0, 100), 
    'delta_m': (0, 1), 
    's_m': (-100, 0)
}
bounds = [params_searching_bounds['p'], params_searching_bounds['g_max'], params_searching_bounds['E_rev'], params_searching_bounds['a_m'], params_searching_bounds['b_m'], params_searching_bounds['delta_m'], params_searching_bounds['s_m']]

In [107]:
# def callback_function(xk, convergence):
#     callback_function.iteration += 1
#     print("Iteration:", callback_function.iteration)
#     print("Solution:", xk)
#     print("Objective value:", obj(xk, sim_setup_2d, target_params))
#     print("------------------------")
# callback_function.iteration = 0  # Initialize the iteration counter

In [244]:
from scipy.optimize import differential_evolution

result = differential_evolution(obj, bounds, args=(sim_setup_2d, target_params), mutation=0.5, recombination=0.9, popsize=5 ,strategy='rand1bin', init='latinhypercube', seed=42, maxiter=1000, disp=True)#, callback=callback_function)

differential_evolution step 1: f(x)= 3.59199e+08
differential_evolution step 2: f(x)= 3.29781e+08
differential_evolution step 3: f(x)= 2.40572e+08
differential_evolution step 4: f(x)= 2.35406e+08
differential_evolution step 5: f(x)= 2.35406e+08
differential_evolution step 6: f(x)= 2.18482e+08
differential_evolution step 7: f(x)= 1.83283e+08
differential_evolution step 8: f(x)= 1.83283e+08
differential_evolution step 9: f(x)= 1.83283e+08
differential_evolution step 10: f(x)= 1.83283e+08
differential_evolution step 11: f(x)= 1.83283e+08
differential_evolution step 12: f(x)= 1.34336e+08
differential_evolution step 13: f(x)= 1.34336e+08
differential_evolution step 14: f(x)= 1.34336e+08
differential_evolution step 15: f(x)= 1.34336e+08
differential_evolution step 16: f(x)= 7.77547e+07
differential_evolution step 17: f(x)= 7.77547e+07
differential_evolution step 18: f(x)= 7.77547e+07
differential_evolution step 19: f(x)= 7.77547e+07
differential_evolution step 20: f(x)= 7.77547e+07
different

/Users/maxwellyue/Documents/GitHub/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:42: RuntimeWarning: invalid value encountered in divide
  return ((tau_0m * np.exp(self.delta_m * ((V - self.V_2m) / self.s_m))) / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))


differential_evolution step 69: f(x)= 7.88752e+06
differential_evolution step 70: f(x)= 7.88752e+06
differential_evolution step 71: f(x)= 7.88752e+06
differential_evolution step 72: f(x)= 7.88752e+06
differential_evolution step 73: f(x)= 7.88752e+06
differential_evolution step 74: f(x)= 7.88752e+06
differential_evolution step 75: f(x)= 7.30456e+06
differential_evolution step 76: f(x)= 7.30456e+06
differential_evolution step 77: f(x)= 5.96276e+06
differential_evolution step 78: f(x)= 5.96276e+06
differential_evolution step 79: f(x)= 5.96276e+06
differential_evolution step 80: f(x)= 5.51195e+06
differential_evolution step 81: f(x)= 5.51195e+06
differential_evolution step 82: f(x)= 5.35188e+06
differential_evolution step 83: f(x)= 4.96627e+06
differential_evolution step 84: f(x)= 4.96627e+06
differential_evolution step 85: f(x)= 3.61853e+06
differential_evolution step 86: f(x)= 3.61853e+06
differential_evolution step 87: f(x)= 3.61853e+06
differential_evolution step 88: f(x)= 3.61853e+06


In [245]:
target_params, result.x

(array([  4.        , 117.62001   , -91.75146   ,   3.0627584 ,
          2.1278825 ,   0.97711813, -10.904842  ], dtype=float32),
 array([  3.99635014, 129.05266701, -79.72431675,   3.09290431,
          2.09275406,   0.96068437, -10.75149895]))

In [243]:
np.mean((target_params - result.x) ** 2)

48.15541751163712

In [110]:
result.x

array([  3.99906788, 116.82382154, -92.6402802 ,   3.06127017,
         2.13322031,   0.97710642, -10.90379333])

In [167]:
result

 message: Maximum number of iterations has been exceeded.
 success: False
     fun: 26.620414408339457
       x: [ 3.295e+00  1.067e+02 -7.884e+01  4.578e+00  1.433e+01
            1.639e-01 -4.363e+01]
     nit: 300
    nfev: 31797
     jac: [ 4.463e-02 -3.375e-04  3.823e-04  0.000e+00  0.000e+00
            0.000e+00  0.000e+00]

### test all samples using vectorized (Grid Search)

In [36]:
dataset = Traces_Dataset('../dataset_test.csv')

params = dataset.params.numpy()
current_traces = dataset.current_traces.numpy()
time_traces = dataset.time_traces.numpy()

prestep_V_2d_vec = dataset.prestep_V.numpy().reshape(-1,1)
step_Vs_2d_vec = dataset.step_Vs.numpy().reshape(-1,1)

In [44]:
def obj(x, *args): 
    '''
    x: a 1-D array of the variables for the obj function (the parameters we are estimating)
    *args=(sim_setup_2d, target_current_trances)
    '''
    trail_model = de_obj_model(x, args[0])
    trail_traces = trail_model.simulation()
    # print(trail_traces[1])
    target_model = de_obj_model(args[1], args[0])
    target_traces = target_model.simulation()
    # print(target_traces[1]) 

    fit = np.sum(np.square(trail_traces - target_traces))
    # relative_error = fit/np.sum(np.square(target_traces))
    
    return fit

In [45]:
# these bounds are from the distribution of the params in the dataset used for NN training
params_searching_bounds = {
    'p': (1, 4),
    'g_max': (100, 140), 
    'E_rev': (-100, -60), 
    'a_m': (0, 100), 
    'b_m': (0, 100), 
    'delta_m': (0, 1), 
    's_m': (-100, 0)
}
bounds = [params_searching_bounds['p'], params_searching_bounds['g_max'], params_searching_bounds['E_rev'], params_searching_bounds['a_m'], params_searching_bounds['b_m'], params_searching_bounds['delta_m'], params_searching_bounds['s_m']]

In [46]:
hyperparameters_grid = {
    'strategy': ['best1bin', 'best1exp', 'rand1exp', 'rand1exp', 
                'rand2bin', 'rand2exp', 'best2bin', 'best2exp',
                'randtobest1bin', 'randtobest1exp',
                'currenttobest1bin', 'currenttobest1exp'],
    'popsize': [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],  # Example popsize hyperparameter
    'mutation': [0.5, (0.1, 0.9)],  # Example mutation hyperparameter
    'recombination': [0.1, 0.2, 0.3,0.4, 0.5, 0.6, 0.7, 0.8, 0.9],  # Example recombination hyperparameter
    'init': ['random', 'sobol', 'latinhypercube'],  # Example init hyperparameter
}


In [47]:
csv_filename = "de_experiment_results_try_2.csv"
# Define the headers for the CSV file
csv_headers = ['Strategy', 'Popsize', 'Mutation', 'Recombination', 'Init', 'MSE Overall Avg', 'MSE Overall Std', 'Elapsed Time Avg', 'Elapsed Time Std']

# Check if the CSV file exists; if not, create and write the headers
if not os.path.exists(csv_filename):
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(csv_headers)

In [48]:
cpu_count()

64

In [49]:
def process_sample(sample, strategy, popsize, mutation, recombination, init):
    prestep_V_2d = prestep_V_2d_vec[sample]
    step_Vs_2d = step_Vs_2d_vec[sample]
    t = time_traces[sample]
    # target_traces = current_traces[sample]
    target_params = params[sample]

    # sim setup for obj evaluation model
    sim_setup_2d = {'prestep_V': prestep_V_2d, 'step_Vs': step_Vs_2d, 't': t}   

    start_time = time.time()
    result = differential_evolution(obj, bounds, args=(sim_setup_2d, target_params), strategy=strategy, popsize=popsize, mutation=mutation, recombination=recombination, init=init, seed=42, maxiter=300)
    end_time = time.time()
    
    mse = (target_params - result.x) ** 2
    elapsed_time = end_time - start_time
    return sample, mse, elapsed_time

if __name__ == '__main__':
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        # pool = Pool()  # Creates a pool of processes
        num_processes = cpu_count()
        
        # Create the pool with the specified number of processes
        pool = Pool(processes=num_processes)  
        
        for strategy in hyperparameters_grid['strategy']:
            for popsize in hyperparameters_grid['popsize']:
                for mutation in hyperparameters_grid['mutation']:
                    for recombination in hyperparameters_grid['recombination']:
                        for init in hyperparameters_grid['init']:
                            # Use partial to fix hyperparameters for the current loop iteration
                            process_func = partial(process_sample, strategy=strategy, popsize=popsize, mutation=mutation, recombination=recombination, init=init)
                            
                            # Map the process function to the sample range using the multiprocessing pool
                            results = pool.map(process_func, range(1000))

                            mse_list = []
                            time_list = []
                            for result in results:
                                sample, mse, elapsed_time = result
                                mse_list.append(mse)
                                time_list.append(elapsed_time)

                            mse_mat = np.vstack(mse_list)
                            time_mat = np.array(time_list).reshape(-1, 1)

                            mse_overall_avg = np.mean(mse_mat)
                            mse_overall_std = np.std(np.mean(mse_mat, axis=1))
                            time_overall_avg = np.mean(time_mat)
                            time_overall_std = np.std(time_mat)
                            
                            writer.writerow([strategy, popsize, mutation, recombination, init, mse_overall_avg, mse_overall_std, time_overall_avg, time_overall_std])
        
        pool.close()  # Close the pool
        pool.join()   # Wait for all processes to finish before exiting



/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized

KeyboardInterrupt: 

Traceback (most recent call last):
  File "/lustre06/project/6000803/maxwell8/Compute-Canada-Research/ENV/lib/python3.10/site-packages/scipy/optimize/_differentialevolution.py", line 1409, in __next__
    energy = self.func(parameters)
  File "/lustre06/project/6000803/maxwell8/Compute-Canada-Research/ENV/lib/python3.10/site-packages/numpy/core/fromnumeric.py", line 2317, in _any_dispatcher
    def _any_dispatcher(a, axis=None, out=None, keepdims=None, *,
  File "/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py", line 75, in simulation
    self.current_traces = self.find_I(self.step_Vs)
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.10.2/lib/python3.10/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
  File "/cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.10.2/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._a

In [15]:
with open(csv_filename, mode='a', newline='') as file:
    writer = csv.writer(file)
    for strategy in hyperparameters_grid['strategy']:
        for popsize in hyperparameters_grid['popsize']:
            for mutation in hyperparameters_grid['mutation']:
                for recombination in hyperparameters_grid['recombination']:
                    for init in hyperparameters_grid['init']:
                        mse_list = []
                        time_list = []
                        for sample in range(3): 
                            
                            prestep_V_2d = prestep_V_2d_vec[sample]
                            step_Vs_2d = step_Vs_2d_vec[sample]
                            t = time_traces[sample]
                            # target_traces = current_traces[sample]
                            target_params = params[sample]

                            # sim setup for obj evaluation model
                            sim_setup_2d = {'prestep_V': prestep_V_2d, 'step_Vs': step_Vs_2d, 't': t}   

                            start_time = time.time()
                            result = differential_evolution(obj, bounds, args=(sim_setup_2d, target_params), strategy=strategy, popsize=popsize, mutation=mutation, recombination=recombination, init=init, seed=42, maxiter=2)
                            end_time = time.time()
                        
                            mse = (target_params - result.x) ** 2
                            mse_list.append(mse)
                            elapsed_time = end_time - start_time
                            time_list.append(elapsed_time)

                        mse_mat = np.vstack(mse_list)
                        time_mat = np.array(time_list).reshape(-1, 1)

                        mse_overall_avg = np.mean(mse_mat)
                        mse_overall_std = np.std(np.mean(mse_mat, axis=1))
                        time_overall_avg = np.mean(time_mat)
                        time_overall_std = np.std(time_mat)
                        
                        writer.writerow([strategy, popsize, mutation, recombination, init, sample, mse_overall_avg, mse_overall_std, time_overall_avg, time_overall_std])

/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:63: RuntimeWarning: divide by zero encountered in divide
  return self.m_infty(V) + (self.m_infty(self.prestep_V) - self.m_infty(V)) * np.exp(- self.t / self.tau_m(V))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:63: RuntimeWarning: invalid value encountered in divide
  return self.m_infty(V) + (self.m_infty(self.prestep_V) - self.m_infty(V)) * np.exp(- self.t / self.tau_m(V))


KeyboardInterrupt: 

### Parallization

In [9]:
import csv
import os
csv_filename = "de_experiment_results_parrallell_4.csv"
# Define the headers for the CSV file
csv_headers = ['Strategy', 'Popsize', 'Mutation', 'Recombination', 'Init', 'MSE Overall Avg', 'MSE Overall Std', 'Elapsed Time Avg', 'Elapsed Time Std']

# Check if the CSV file exists; if not, create and write the headers
if not os.path.exists(csv_filename):
    with open(csv_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(csv_headers)

In [11]:
import csv
import time
import numpy as np
from functools import partial
from multiprocessing import Pool
from scipy.optimize import differential_evolution

def process_sample(sample, strategy, popsize, mutation, recombination, init):
    prestep_V_2d = prestep_V_2d_vec[sample]
    step_Vs_2d = step_Vs_2d_vec[sample]
    t = time_traces[sample]
    # target_traces = current_traces[sample]
    target_params = params[sample]

    # sim setup for obj evaluation model
    sim_setup_2d = {'prestep_V': prestep_V_2d, 'step_Vs': step_Vs_2d, 't': t}   

    start_time = time.time()
    result = differential_evolution(obj, bounds, args=(sim_setup_2d, target_params), strategy=strategy, popsize=popsize, mutation=mutation, recombination=recombination, init=init, seed=42, maxiter=2)
    end_time = time.time()
    
    mse = (target_params - result.x) ** 2
    elapsed_time = end_time - start_time
    return sample, mse, elapsed_time

if __name__ == '__main__':
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        pool = Pool()  # Creates a pool of processes
        
        for strategy in hyperparameters_grid['strategy']:
            for popsize in hyperparameters_grid['popsize']:
                for mutation in hyperparameters_grid['mutation']:
                    for recombination in hyperparameters_grid['recombination']:
                        for init in hyperparameters_grid['init']:
                            # Use partial to fix hyperparameters for the current loop iteration
                            process_func = partial(process_sample, strategy=strategy, popsize=popsize, mutation=mutation, recombination=recombination, init=init)
                            
                            # Map the process function to the sample range using the multiprocessing pool
                            results = pool.map(process_func, range(3))

                            mse_list = []
                            time_list = []
                            for result in results:
                                sample, mse, elapsed_time = result
                                mse_list.append(mse)
                                time_list.append(elapsed_time)

                            mse_mat = np.vstack(mse_list)
                            time_mat = np.array(time_list).reshape(-1, 1)

                            mse_overall_avg = np.mean(mse_mat)
                            mse_overall_std = np.std(np.mean(mse_mat, axis=1))
                            time_overall_avg = np.mean(time_mat)
                            time_overall_std = np.std(time_mat)
                            
                            writer.writerow([strategy, popsize, mutation, recombination, init, mse_overall_avg, mse_overall_std, time_overall_avg, time_overall_std])
        
        pool.close()  # Close the pool
        pool.join()   # Wait for all processes to finish before exiting


/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized_exp_setup/DE/DE_obj_model.py:33: RuntimeWarning: overflow encountered in exp
  return (1 / (1 + np.exp((V - self.V_2m) / self.s_m))).reshape((-1,1))
/lustre06/project/6000803/maxwell8/Compute-Canada-Research/Exp_rates_generalized

KeyboardInterrupt: 

In [257]:
np.mean(mse_mat, axis=1).shape

(439,)

In [255]:
np.mean(mse_mat), np.std(np.mean(mse_mat, axis=1)), np.mean(time_mat), np.std(time_mat)

(511.3364727433909, 417.878264540534, 0.1499422720733156, 0.033234656515269134)

In [252]:
np.mean(np.mean(mse_mat, axis=0)), np.mean(time_mat, axis=0)

(511.336472743391, array([0.14994227]))

In [249]:
time_mat.shape, mse_mat.shape

((439, 1), (439, 7))

In [159]:
np.hstack((mse_mat, time_array.reshape(-1, 1)))

(220, 8)

#### Run DE on all test dataset, check result mse on all params here

In [16]:
mse_mat = np.load('de_mse_params_all_test.npy')

In [17]:
mse_mat.shape

(30, 7)

In [18]:
mse_params = np.mean(mse_mat, axis=0)
mse_params

array([1.68353114e-03, 5.70682053e+01, 1.12901201e+02, 2.79381230e-03,
       7.58826951e+00, 1.83145674e-04, 7.30552287e-02])

## Define obj fct and estimate for single param

In [16]:
def obj_i_param(x, *args): 
    '''
    x: a single float value of the i-th param
    args: a tupleo f additional fixed parameters (prestep_V, step_V0, time_traces)
    *args=(i, sim_setup_2d, target_params)
    '''
    params = args[2].copy()
    params[args[0]] = x
    trail_model = de_obj_model(params, args[1])
    trail_traces = trail_model.simulation()
    # print(trail_traces[1])
    target_model = de_obj_model(args[2], args[1])
    target_traces = target_model.simulation()
    # print(target_traces[1]) 
    
    return np.sum(np.square(trail_traces - target_traces))

In [21]:
params_searching_bounds = {
    'p': (1, 5),
    'g_max': (100, 140), 
    'E_rev': (-100, -60), 
    'a_m': (0, 13), 
    'b_m': (0, 100), 
    'delta_m': (0, 1), 
    's_m': (-17, -10)
}
i = 6
key_at_index = list(params_searching_bounds.keys())[i]
bounds = [params_searching_bounds[key_at_index]]
bounds

[(-17, -10)]

In [22]:
from scipy.optimize import differential_evolution
result = differential_evolution(obj_i_param, bounds, args=(i, sim_setup_2d, target_params), maxiter=100)#, tol = 1e-10)
result

 message: Optimization terminated successfully.
 success: True
     fun: 0.0
       x: [-1.105e+01]
     nit: 22
    nfev: 347

In [32]:
((target_params[i] - result.x) ** 2)

array([2.0789837e-15])

#### Results for all single param estimation

In [27]:
mses = np.load('de_mses_on_ith_params.npy')
nits = np.load('de_nit_on_ith_params.npy')
mses.shape, nits.shape

((30, 7), (30, 7))

In [28]:
arr = mses.mean(axis=0)
print(arr)

sorted_indices = np.argsort(arr)

print("Sorted indices:", sorted_indices)

[1.40752475e-15 6.77700377e-15 2.20245039e-15 1.40308669e-15
 2.14625476e-15 1.44141350e-15 2.20503233e-15]
Sorted indices: [3 0 5 4 2 6 1]


In [29]:
arr = nits.mean(axis=0)
print(arr)

sorted_indices = np.argsort(arr)

print("Sorted indices:", sorted_indices)

[27.         26.1        26.53333333 26.53333333 26.83333333 27.23333333
 26.33333333]
Sorted indices: [1 6 2 3 4 0 5]
